# ARIM-Academy：　機器データ利活用ユースケース（１）
### 機器：走査電子顕微鏡（SEM）
### 分析：CNNによる画像識別　（データ増幅編）

## データセット

**『LIBカソード電極のSEMデータセット』**は、さまざまなLi（Ni、Co、Mn）O 2（NCM）カソードの主な組成（例：333、523、622、および811）とさまざまな状態（例：初期、サイクル前、および100回サイクル）を分類する予測モデルを開発を目的に集められた走査電子顕微鏡（SEM）画像です。

[1] Oh, J., Yeom, J., Madika, B. et al. Composition and state prediction of lithium-ion cathode via convolutional neural network trained on scanning electron microscopy images. npj Comput Mater 10, 88 (2024). https://doi.org/10.1038/s41524-024-01279-6

<img src="./img/image.png" alt="メインイメージ">

### コードへの接続
google colabにおけるオンラインの場合にこのラインを実行します。（<font color="red">Google colabに接続しない場合には不要</font>）

In [ ]:
!git clone https://github.com/ARIM-Usecase/Example_4.git
%cd Example_4/code

### ライブラリのインポート
コード実行で必要なpythonのライブラリをimport文でロードします。

In [8]:
# I/Oライブラリ
import os
import random
import sys

# 汎用イブラリ
from PIL import Image

### データアーギュメント（データ増幅化）

In [9]:
# Directory function
def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [10]:
# Function to generate images
def generate_imgs(label, imgs, size, pixel=True, path=None, verbose=False):
    if len(imgs) == 0:  # Check if the image list is empty
        print(f"No images found for {label}. Skipping...")
        return

    if pixel == True and verbose:
        print('Generated image size will be half of source image size.')

    save_path = make_dir(os.path.join(path))

    for _ in range(size):
        file_num = random.randint(0, len(imgs) - 1)
        img_name, img_format = os.path.splitext(imgs[file_num])

        image = Image.open(os.path.join('data', label, imgs[file_num]))

        if pixel == 'full':
            generated_img_name = f'{label}_full_{img_name}{img_format}'

            # y-axis > 895 pixel for removing information of SEM images
            image.crop((0, 0, image.size[0], image.size[1] - 65)).save(os.path.join(save_path, generated_img_name))

        else:
            # Set the size of generated images
            pixels = [int(image.size[0] / 2), int(image.size[1] / 2)] if pixel == True else [int(pixel), int(pixel)]  # [size, size] / square shape

            if image.size[0] - pixels[0] < 0 and image.size[1] - pixels[1] < 0:
                print('Cropped images must be smaller than full-sized images.')
                sys.exit(1)

            # Generate random upper-left coordinate
            x_coord, y_coord = random.randint(0, image.size[0] - pixels[0]), random.randint(0, image.size[1] - pixels[1] - 65)

            # Cropping
            generated_img_name = f'{label}_x{x_coord}y{y_coord}_{img_name}{img_format}'
            image.crop((x_coord, y_coord, x_coord + pixels[0], y_coord + pixels[1])).save(os.path.join(save_path, generated_img_name))

    if verbose:
        print(f'Saved path: {path}')

### 初期設定

In [11]:
# Variables
path = 'data'  # Set the folder path (data folder is at the same level as the notebook)
size = 100  # Set the desired dataset size
pixel = True  # Set pixel size for generated images
test_ratio = 10  # Ratio for test dataset
validation_ratio = 20  # Ratio for validation dataset
name = None  # Dataset name
seed = 1345879  # Seed for reproducibility
verbose = True  # Set verbosity

### ファイル操作

In [12]:
# Step 1: Prepare paths and labels
full_path = os.path.join('./', path)
labels = [label for label in os.listdir(full_path) if os.path.isdir(os.path.join(full_path, label))]

In [13]:
# Step 2: Determine save path
if name:
    save_path = os.path.join('./data', name)
else:
    p = pixel if pixel != True else 'Half'
    save_path = os.path.join(f'./data/size{size}_pixel{p}')

In [15]:
# Step 3: Process each label
for label in labels:
    # Get image file list (change 'jpg' extension if needed)
    imgs = [f for f in os.listdir(os.path.join(full_path, label)) if f[-3:] == 'jpg' and f[0] != '.']
    num_imgs = len(imgs)

    if verbose:
        print(f'Number of images in {label} folder: {num_imgs}')

    # Step 4: Split the dataset into train, validation, and test sets
    num_test = int(round(test_ratio / 100 * num_imgs, 0))
    num_imgs -= num_test
    num_validate = int(round(validation_ratio / 100 * num_imgs, 0))
    num_train = num_imgs - num_validate

    if verbose:
        print(f'Number of training {label} images: {num_train}')
        print(f'Number of validation {label} images: {num_validate}')
        print(f'Number of test {label} images: {num_test}')

    # Step 5: Generate full images if size == 0
    if size == 0:
        generate_imgs(label, imgs, len(imgs), pixel='full', path=os.path.join(full_path, 'test_full'))

    else:
        # Seed for randomness consistency
        random.seed(seed)

        # Step 6: Randomly select test images
        test_imgs = []
        for _ in range(num_test):
            if len(imgs) > 0:
                test_imgs.append(imgs.pop(random.randint(0, len(imgs) - 1)))

        # Step 7: Randomly select validation images
        validate_imgs = []
        for _ in range(num_validate):
            if len(imgs) > 0:
                validate_imgs.append(imgs.pop(random.randint(0, len(imgs) - 1)))

        # Remaining images are for training
        training_imgs = imgs

        # Step 8: Generate test images
        generate_imgs(label, test_imgs, size=int(size * test_ratio / 100),
                      pixel=pixel, path=os.path.join(save_path, 'test', label), verbose=verbose)

        # Step 9: Generate training images
        generate_imgs(label, training_imgs, int(size * (1 - validation_ratio / 100)),
                      pixel=pixel, path=os.path.join(save_path, 'train', label), verbose=verbose)

        # Step 10: Generate validation images
        generate_imgs(label, validate_imgs, int(size * validation_ratio / 100),
                      pixel=pixel, path=os.path.join(save_path, 'validate', label), verbose=verbose)

if verbose:
    print('Done!')

Number of images in 333 folder: 183
Number of training 333 images: 132
Number of validation 333 images: 33
Number of test 333 images: 18
Generated image size will be half of source image size.
Saved path: ./data/size100_pixelHalf\test\333
Generated image size will be half of source image size.
Saved path: ./data/size100_pixelHalf\train\333
Generated image size will be half of source image size.
Saved path: ./data/size100_pixelHalf\validate\333
Number of images in 333_cycled folder: 82
Number of training 333_cycled images: 59
Number of validation 333_cycled images: 15
Number of test 333_cycled images: 8
Generated image size will be half of source image size.
Saved path: ./data/size100_pixelHalf\test\333_cycled
Generated image size will be half of source image size.
Saved path: ./data/size100_pixelHalf\train\333_cycled
Generated image size will be half of source image size.


FileNotFoundError: [Errno 2] No such file or directory: './data/size100_pixelHalf\\validate\\333_cycled\\333_cycled_x265y358_ETRI_ 20 kV_SE_HighVac_x500 __573.jpg'